In [1]:
import yaml
import os
import psycopg2
import sys
import json
import csv
import mysql.connector

/home/adrianekas/.local/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
DIR = os.getcwd()
CONFIG_PATH = DIR + '/config/'

APP = CONFIG_PATH + 'app.yaml'
with open(APP, 'r') as stream:
    try:
        APP_CONFIG = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

print APP_CONFIG
ENV = APP_CONFIG['env']
VERSION = APP_CONFIG['version']

{'version': '1.0.0', 'env': 'development'}


In [3]:
FORMULA = CONFIG_PATH + 'formula.yaml'
with open(FORMULA, 'r') as stream:
    try:
        FORMULA_CONFIG = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

print len(FORMULA_CONFIG['formula'])

285


In [4]:
class Stack:
     def __init__(self):
         self.items = []

     def isEmpty(self):
         return self.items == []

     def push(self, item):
         self.items.append(item)

     def pop(self):
         return self.items.pop()

     def peek(self):
         return self.items[len(self.items)-1]

     def size(self):
         return len(self.items)

In [5]:
stack = Stack()
postfix_string = ""

# Prepare Connection

In [6]:
ML_DATABASE = CONFIG_PATH + 'ml.database.yaml'
with open(ML_DATABASE, 'r') as stream:
    try:
        CONFIG = yaml.load(stream)
        ML_CONFIG_ENV = CONFIG[ENV]
    except yaml.YAMLError as exc:
        print(exc)

mydb = mysql.connector.connect(
  host=ML_CONFIG_ENV['host'],
  port=ML_CONFIG_ENV['port'],
  user=ML_CONFIG_ENV['username'],
  passwd=ML_CONFIG_ENV['password'],
  database=ML_CONFIG_ENV['database']
)

# Spliting Elements

In [7]:
formula = FORMULA_CONFIG['formula']
elements = formula.split(" ")

# Insert Data

In [8]:
def getPriority(char):
    if (char == "+" or char == "-"):
        return 1 #lower priority
    elif (char == "*" or char == "/"):
        return 2 #higher priority

In [9]:
def is_it_operand(element):
    if (element != "(" and element != ")" and element != "-" and element != "+" and element != "/" and element != "*"):
        return True
    else:
        return False

In [10]:
def pop_all_element_until_find_open_bracket():
    global postfix_string
    while True:
        if stack.peek() == "(":
            stack.pop()
            return
        else:
            postfix_string += stack.pop() + " "

In [11]:
def pop_all():
    global postfix_string
    for i in range (0,stack.size()):
        postfix_string += stack.pop() + " "

In [12]:
#elements = "4 + 6 * ( 5 - 2 ) / 3"
for e in elements:
    if is_it_operand(e):
        postfix_string += e + " "
    else:
        if e == "(":
            stack.push(e)
            continue
        elif e == ")":
            pop_all_element_until_find_open_bracket()
            continue
            
        if not stack.isEmpty():
            if getPriority(stack.peek()) >= getPriority(e):
                postfix_string += stack.pop() + " "
        stack.push(e)

In [13]:
pop_all()

In [14]:
print postfix_string

TIPE_PERUSAHAAN W_TIPE_PERUSAHAAN * PENDIDIKAN_TERAKHIR W_PENDIDIKAN_TERAKHIR * + LOCATION W_LOCATION * + TIPE_INDUSTRI W_TIPE_INDUSTRI * + KATEGORI_PEKERJAAN W_KATEGORI_PEKERJAAN * + JENJANG_KARIR W_JENJANG_KARIR * + PENGALAMAN_KERJA W_PENGALAMAN_KERJA * + 


# Update Data In Database

In [17]:
def insertFormulaData(formula,postfix_string):
    sql = ("INSERT INTO Salary_Formula (versi, "
           "infix, postfix)" +
           "VALUES (%s,%s,%s)")
    mycursor = mydb.cursor()
    val = (VERSION,formula,postfix_string)
    mycursor.execute(sql, val)
    mydb.commit()

In [18]:
insertFormulaData(formula,postfix_string)